In [38]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
matplotlib.use('Qt5Agg') 
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from qutip import *

import src.encoding.evolution as evolution
import importlib
importlib.reload(evolution)
from src.encoding.evolution import Evolve
from tqdm import tqdm
from src.helper_functions import *

In [35]:

fidelity(thermal_dm(10,0), fock_dm(10,0))

1.0

In [ ]:
def vectorize(y):
    I = np.eye(10)
    arr = []
    for i in y: arr.append(I[i])
    return np.column_stack(arr)
def lreg(X, y):
    x = X.T
    X1 = np.vstack((np.ones(len(x[0])), x))
    cM = vectorize(y)@np.linalg.pinv(X1)
    c = cM[:, 0]
    c = c[..., None]
    M = cM[:, 1:]
    return M, c



In [3]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)


In [4]:
def reshape(image, label):
    #image = tf.image.resize(image, [n, n], method='gaussian', antialias=True)
    image = tf.cast(image, tf.float32) / 255
    return tf.reshape(image, [-1]), label


ds_train = ds_train.map(reshape, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.map(reshape, num_parallel_calls=tf.data.AUTOTUNE)

In [5]:
trainx = []
trainy= []
testx = []
testy = []
for i, l in ds_train.take(60000):
    trainx.append(i.numpy())
    trainy.append(l.numpy())
for i, l in ds_test.take(10000):
    testx.append(i.numpy())
    testy.append(l.numpy())

In [7]:
trainx = np.array(trainx)
trainy = np.array(trainy)
testx = np.array(testx)
testy = np.array(testy)

In [8]:
from sklearn.decomposition import PCA
pca = PCA(16)
x_pca = pca.fit_transform(np.vstack((trainx, testx)))

In [9]:
from sklearn.preprocessing import normalize
x_pca = normalize(x_pca)

In [10]:
testx_pca = x_pca[60000:, :]
trainx_pca = x_pca[:60000, :]

In [99]:
import src.encoding.evolution as evolution
import importlib
importlib.reload(evolution)
from src.encoding.evolution import Evolve

In [59]:
img = trainx_pca[12199]
splits = 10
x = .175e6
tt = 18
cdim = 30
xaxis = 20
e = Evolve(img, measurement='pns trace', cdim=cdim, dm=10, t_total=tt*1e-6, anharm=False, c_ops=True,
                      ffilter=False, intervals=splits, nsteps=5000, pc_imag=x, pq_imag=x)
e.vector('2qubit')

rhos = e.rho_test()
for rho in rhos:
    rho = rho.ptrace(1)
    photons =[(fock_dm(cdim, i)*rho).tr() for i in range(xaxis)]
    
    plt.plot(list(range(xaxis)), photons)
plt.legend([f"t = {np.round(k,2)}e-6" for k in np.linspace(0,tt,splits+1)[1:]])
plt.show()

KeyboardInterrupt: 

In [126]:
import src.encoding.evolution as evolution
import importlib
importlib.reload(evolution)
from src.encoding.evolution import Evolve

In [131]:
img = trainx_pca[12119]
splits = 3
x = .175e6
tt = 18
cdim = 13
e = Evolve(img, measurement='pns trace', cdim=cdim, dm=10, t_total=tt*1e-6, anharm=False, c_ops=True,
                      ffilter=False, intervals=splits, nsteps=5000, pc_imag=x, pq_imag=x)
r1 = np.array(e.vector('2qubit'))
e = Evolve(img, measurement='pns', cdim=cdim, dm=10, t_total=tt*1e-6, anharm=False, c_ops=True,
                      ffilter=False, intervals=splits, nsteps=5000, pc_imag=x, pq_imag=x)
r2 = np.array(e.vector('2qubit'))


In [132]:
plt.plot(r1)
plt.plot(r2)
plt.show()

In [133]:
np.mean(r1-r2)

-0.0002010437140480386

In [134]:
np.std(r1-r2)

6.37560669337563e-05

In [0]:

fig, axes = plt.subplots(3, 3, figsize=(7, 7))

axes = np.array(axes).reshape(-1)
for i in range(9):
    pixels = trainx[i].reshape(28,28)
    axes[i].imshow(pixels, cmap='gray')
    axes[i].set_title(f"{trainy[i]}")
    axes[i].axis('off')

In [ ]:
from sklearn.linear_model import RidgeClassifier, LogisticRegression
clf = RidgeClassifier(alpha=0)
clf.fit(trainx_pca, trainy)
clf.score(testx_pca, testy)

In [ ]:
train_noise = np.random.normal(0, 1e-3, size=trainx_pca.shape)
test_noise = np.random.normal(0, 1e-3, size=testx_pca.shape)
clf = LogisticRegression(max_iter=1000)
clf.fit(trainx_pca + train_noise, trainy)
clf.score(testx_pca + test_noise, testy)

In [ ]:
M, C = lreg(trainx_pca, trainy)
M

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(5, 5))

axes = np.array(axes).reshape(-1)
for i in range(9):
    pixels = trainx_pca[i].reshape(4,4)
    axes[i].imshow(pixels, cmap='gray')
    axes[i].set_title(f"{trainy[i]}")
    axes[i].axis('off')
plt.show()

In [ ]:
from scipy.fftpack import rfft, irfft, fftfreq
def freq_filter(signal):
    signal = np.repeat(signal,100)
    #signal = np.pad(signal, (5, 5), 'constant')
    W = fftfreq(signal.size, d=0.001)
    f_signal = rfft(signal)
    cut_f_signal = f_signal.copy()
    cut_f_signal[(abs(W) > 12)] = 0
    cut_signal = irfft(cut_f_signal)
    return cut_signal

In [ ]:
plt.plot(freq_filter(list(trainx_pca[3])), color='r')
plt.ylim(-1,1)
plt.show()

In [ ]:
trainx_pca[3][1]